# Soy Bean
Trata-se de um dataset com dados agronomicos de uma plantação de soja e uma doença que acometeu cada planta.

O objetivo é, com base nos dados, prever qual doença vai afetar cada planta.

A tecnina utilizada será Rede Neural com a biblioteca Keras

## Bibliotecas e leitura dos dados

In [2]:
import pandas as pd # Modelagem de dados
import numpy as np # Algebra Linear

from sklearn.compose import make_column_transformer # Transformação em colunas de um DataFrame
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # Encoders para variaveis categoricas
from sklearn.model_selection import train_test_split # Divisão entre teste e treino

from keras.models import Sequential # Rede Neural com comandos sequenciais
from keras.layers import Dense # Camadas

In [3]:
# Le o dataset em formato CSV
data = pd.read_csv('soybean.csv')
data.shape

(683, 36)

In [4]:
# Mostra as 5 primeiras linhas do dataset
data.head()

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,october,normal,gt-norm,norm,yes,same-lst-yr,low-areas,pot-severe,none,90-100,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
1,august,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,severe,fungicide,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
2,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,fungicide,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
3,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,none,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
4,october,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,pot-severe,none,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker


In [5]:
# Exibe a informação de cada coluna presente no dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             683 non-null    object
 1   plant-stand      683 non-null    object
 2   precip           683 non-null    object
 3   temp             683 non-null    object
 4   hail             683 non-null    object
 5   crop-hist        683 non-null    object
 6   area-damaged     683 non-null    object
 7   severity         683 non-null    object
 8   seed-tmt         683 non-null    object
 9   germination      683 non-null    object
 10  plant-growth     683 non-null    object
 11  leaves           683 non-null    object
 12  leafspots-halo   683 non-null    object
 13  leafspots-marg   683 non-null    object
 14  leafspot-size    683 non-null    object
 15  leaf-shread      683 non-null    object
 16  leaf-malf        683 non-null    object
 17  leaf-mild        683 non-null    ob

## Processamento e Modelagem de dados

In [6]:
# Separa os dados entre X = Previsores, e Y = Target
x = data.drop('class', axis=1)
y = data['class']
print('X: {}, Y: {}'.format(x.shape, y.shape))

X: (683, 35), Y: (683,)


In [7]:
# Utiliza tecnica de One Hot Encode no target Y pois há mais que duas classes
y = pd.get_dummies(y)
y.shape

(683, 19)

#### Variáveis Categoricas
Variaveis categoricas podem ser:
    <li>Nominais: Representa um nome, uma categoria, sem ordem definida.
    <li>Ordinais: Possui uma hierarquia, uma ordem.

Através de análise pelo excel, foi identificado as possiveis variáveis Ordinais e Nominais.

In [8]:
ordinal = ['date', 'crop-hist', 'severity', 'germination', 'leafspot-size']
nominal = ['plant-stand', 'precip', 'temp', 'hail', 'area-damaged', 'seed-tmt', 'plant-growth', 'leaves', 'leafspots-halo', 
           'leafspots-marg', 'leaf-shread', 'leaf-malf', 'leaf-mild', 'stem', 'lodging', 'stem-cankers', 'canker-lesion', 
           'fruiting-bodies', 'external-decay', 'mycelium', 'int-discolor', 'sclerotia', 'fruit-pods', 'fruit-spots', 
          'seed', 'mold-growth', 'seed-discolor', 'seed-size', 'shriveling', 'roots']
print(len(ordinal) + len(nominal))

35


In [9]:
x[ordinal]

,date,crop-hist,severity,germination,leafspot-size
0,october,same-lst-yr,pot-severe,90-100,dna
1,august,same-lst-two-yrs,severe,80-89,dna
2,july,same-lst-yr,severe,lt-80,dna
3,july,same-lst-yr,severe,80-89,dna
4,october,same-lst-two-yrs,pot-severe,lt-80,dna
...,...,...,...,...,...
678,april,?,?,?,dna
679,april,diff-lst-year,?,?,dna
680,june,diff-lst-year,?,?,dna
681,april,same-lst-yr,?,?,gt-1/8


### Tecnicas para tratar variaveis categoricas

##### Label Encoder
<p>Label Encoder trata variaveis ordinais, subistituindo seus valores texto (string) por numeros naturais de forma a demonstrar uma certa ordem.
<p>Ex: Columa Mes = ['Jan', 'Fev', 'Mar'] é transformada em Coluna Mes = [1, 2, 3]

##### One Hot Encoder
<p>One Hot Encoder trata variaveis nominais, subistituindo seus valores text (string) por um vetor binário que é distribuido em colunas.
<p>Ex: Coluna Sexo = ['F', 'M'] é transformada em Coluna F = 0, Coluna M = 1

In [10]:
# Aplica Label Encoder as colunas ordinais
for k in ordinal:
    labelEncoder = LabelEncoder()
    x[k] = labelEncoder.fit_transform(x[k])
x.head()

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,int-discolor,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots
0,6,normal,gt-norm,norm,yes,4,low-areas,2,none,1,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
1,2,normal,gt-norm,norm,yes,3,scattered,3,fungicide,0,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
2,3,normal,gt-norm,norm,yes,4,scattered,3,fungicide,3,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
3,3,normal,gt-norm,norm,yes,4,scattered,3,none,0,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
4,6,normal,gt-norm,norm,yes,3,scattered,2,none,3,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm


In [11]:
# Aplica One Hot Encoder nas colunas Nominais
one_hot = make_column_transformer((OneHotEncoder(categories='auto', sparse=False), nominal), remainder='passthrough')
x = pd.DataFrame(one_hot.fit_transform(x))
x.head()

,0,1,2,3,4,5,6,7,8,9,...,103,104,105,106,107,108,109,110,111,112
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,6.0,4.0,2.0,1.0,1.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2.0,3.0,3.0,0.0,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,3.0,4.0,3.0,3.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,3.0,4.0,3.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,6.0,3.0,2.0,3.0,1.0


### Separação em Treino e Teste

<p> É separado as variaveis em Treino e Teste, utilizando uma porcentagem de 70% treino e 30% teste.

In [12]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=0)

## Criação do Modelo

<p> O modelo sera uma Rede Neural utilizando Keras e a classe Sequential()
<p> A camada de entrada é de tamanho 113, referente ao numero de colunas dos dados processados.
<p> As camadas intermediarias possuem tamanho 66 e 43. De acordo com a formula (entrada + saida)/2 que neste caso é (113+19)/2 e (66+19)/2 respectivamente

In [15]:
model = Sequential()
model.add(Dense(units=66, input_dim=113))
model.add(Dense(units=43))
model.add(Dense(units=19, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

<p> O Modelo é treinado por 300 epocas.

In [16]:
model.fit(x_treino, y_treino, epochs=300)

Epoch 1/300
15/15 [==============================] - 6s 3ms/step - loss: 0.6865 - accuracy: 0.1154
Epoch 2/300
15/15 [==============================] - 0s 3ms/step - loss: 0.2303 - accuracy: 0.1952
Epoch 3/300
15/15 [==============================] - 0s 3ms/step - loss: 0.1689 - accuracy: 0.3277
Epoch 4/300
15/15 [==============================] - 0s 3ms/step - loss: 0.1471 - accuracy: 0.4859
Epoch 5/300
15/15 [==============================] - 0s 3ms/step - loss: 0.1299 - accuracy: 0.5437
Epoch 6/300
15/15 [==============================] - 0s 3ms/step - loss: 0.1184 - accuracy: 0.6154
Epoch 7/300
15/15 [==============================] - 0s 3ms/step - loss: 0.1092 - accuracy: 0.6653
Epoch 8/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0964 - accuracy: 0.7414
Epoch 9/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0911 - accuracy: 0.7636
Epoch 10/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0844 - accuracy: 0.7672
Epoch 11/

15/15 [==============================] - 0s 2ms/step - loss: 0.0155 - accuracy: 0.9323
Epoch 84/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0165 - accuracy: 0.9261
Epoch 85/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0147 - accuracy: 0.9410
Epoch 86/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0188 - accuracy: 0.9250
Epoch 87/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0164 - accuracy: 0.9281
Epoch 88/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0165 - accuracy: 0.9299
Epoch 89/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0177 - accuracy: 0.9114
Epoch 90/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0157 - accuracy: 0.9399
Epoch 91/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0184 - accuracy: 0.9145
Epoch 92/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0161 - accuracy: 0.9365
Epoch 93/300


Epoch 164/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0155 - accuracy: 0.9286
Epoch 165/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 0.9394
Epoch 166/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0152 - accuracy: 0.9463
Epoch 167/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0121 - accuracy: 0.9563
Epoch 168/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 0.9363
Epoch 169/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0159 - accuracy: 0.9239
Epoch 170/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0209 - accuracy: 0.9124
Epoch 171/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0119 - accuracy: 0.9481
Epoch 172/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0124 - accuracy: 0.9447
Epoch 173/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0147 - accurac

15/15 [==============================] - 0s 3ms/step - loss: 0.0128 - accuracy: 0.9436
Epoch 246/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0139 - accuracy: 0.9407
Epoch 247/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0124 - accuracy: 0.9429
Epoch 248/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0128 - accuracy: 0.9479
Epoch 249/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0139 - accuracy: 0.9466
Epoch 250/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0133 - accuracy: 0.9394
Epoch 251/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0113 - accuracy: 0.9599
Epoch 252/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0136 - accuracy: 0.9522
Epoch 253/300
15/15 [==============================] - 0s 2ms/step - loss: 0.0138 - accuracy: 0.9401
Epoch 254/300
15/15 [==============================] - 0s 3ms/step - loss: 0.0124 - accuracy: 0.9508
Epoc

Resultado com 113 Na camada de entrada, 66 na primeira camada oculta, 33 na segunda camada oculta, 19 na camada de saida 
e 300 epocas: 94%

<img src='rede.png'>

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 66)                7524      
_________________________________________________________________
dense_4 (Dense)              (None, 43)                2881      
_________________________________________________________________
dense_5 (Dense)              (None, 19)                836       
Total params: 11,241
Trainable params: 11,241
Non-trainable params: 0
_________________________________________________________________
